In [ ]:
import pandas as pd
import re

try:
    df = pd.read_csv('final_food_delivery_dataset.csv')
except:
    # Fallback to regeneration
    orders_df = pd.read_csv('orders.csv')
    users_df = pd.read_json('users.json')
    restaurant_data = []
    with open('restaurants.sql', 'r') as f:
        for line in f:
            if line.strip().startswith("INSERT INTO restaurants VALUES"):
                match = re.search(r"\((.*)\);", line)
                if match:
                    restaurant_data.append([p.strip().strip("'") for p in match.group(1).split(',')])
    restaurants_df = pd.DataFrame(restaurant_data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])
    restaurants_df['restaurant_id'] = pd.to_numeric(restaurants_df['restaurant_id'])
    merged = pd.merge(orders_df, users_df, on='user_id', how='left')
    df = pd.merge(merged, restaurants_df, on='restaurant_id', how='left', suffixes=('_order', '_details'))
    df['rating'] = pd.to_numeric(df['rating'])

# Filter for Gold members
gold_members = df[df['membership'] == 'Gold']

# Group by city and calculate average order value
city_avg_order_value_gold = gold_members.groupby('city')['total_amount'].mean().sort_values(ascending=False)

print(city_avg_order_value_gold)
print(f"\nCity with highest average order value (Gold): {city_avg_order_value_gold.idxmax()}")
print(f"Average Value: {city_avg_order_value_gold.max()}")